In [1]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy 
import random
import requests
from config import weather_api_key
from datetime import datetime
import json
import time
import gmaps
from config import g_key

In [2]:
#Avoid error when loading heatmaps from gmaps
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [3]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [73]:
#Import data
vacation_search_df=pd.read_csv('data/WeatherPy_vacation.csv')
vacation_search_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,78.80,clear sky,19.73,-155.09,Hilo Hawaiian Hotel
1,Agadez,NE,88.11,clear sky,19.75,10.25,NaN
2,Byron Bay,AU,75.20,scattered clouds,-28.65,153.62,Backpackers Inn on the Beach
3,Lakatoro,VU,84.20,few clouds,-16.10,167.42,Lakatoro Palm Lodge
4,Puerto Maldonado,PE,71.38,overcast clouds,-12.60,-69.18,Wasai Hotel
...,...,...,...,...,...,...,...
221,Zhicheng,CN,88.61,clear sky,30.30,111.50,Wangjiang Building
222,Saint Pete Beach,US,81.00,broken clouds,27.73,-82.74,TradeWinds Island Grand Resort
223,Sandakan,MY,89.60,few clouds,5.84,118.12,Hotel Sandakan
224,Chalus,IR,71.60,few clouds,36.66,51.42,نظام مهندسي ساختمان چالوس


In [75]:
#Choose the vacation destination
travel_df=vacation_search_df.loc[(vacation_search_df['Country']=='AU')]
travel_df.reset_index(drop=True, inplace=True)
travel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Byron Bay,AU,75.20,scattered clouds,-28.65,153.62,Backpackers Inn on the Beach
1,Port Keats,AU,88.88,clear sky,-14.25,129.55,TDC Village (Hundred Man Camp)
2,Roebourne,AU,79.02,overcast clouds,-20.78,117.13,Harding River Caravan Park
3,Port Macquarie,AU,70.00,scattered clouds,-31.43,152.92,Rydges Port Macquarie
4,Carnarvon,AU,77.05,overcast clouds,-24.87,113.63,Hospitality Carnarvon
5,Geraldton,AU,73.40,broken clouds,-28.77,114.60,Broadwater Mariner Resort
6,Charlestown,AU,70.00,light rain,-32.95,151.67,Panorama Motor Inn Hotel
7,Cairns,AU,80.60,scattered clouds,-16.92,145.77,Mantra Trilogy Cairns
8,Port Hedland,AU,84.20,overcast clouds,-20.32,118.57,The Esplanade Hotel
9,Biloela,AU,75.99,overcast clouds,-24.42,150.50,Silo Motor Inn


In [76]:
#Define start city
sydney=travel_df.iloc[10, 4:6]
sydney

Lat    -33.87
Lng    151.21
Name: 10, dtype: object

In [78]:
#Define city - 2
charlestown=travel_df.iloc[6, 4:6]

In [79]:
#Define city - 3
port_macquarie=travel_df.iloc[3,4:6]

In [80]:
#Define city - 4
byron_bay=travel_df.iloc[0,4:6]

In [81]:
#Define city - 5
biloela=travel_df.iloc[9,4:6]

In [91]:
#Create dataframe with cities - to be used to create map with markers
round_trip_df=travel_df.iloc[[10,6,3,0,9]]
round_trip_df.reset_index(drop=True, inplace=True)
round_trip_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sydney,AU,70.00,broken clouds,-33.87,151.21,QT Sydney
1,Charlestown,AU,70.00,light rain,-32.95,151.67,Panorama Motor Inn Hotel
2,Port Macquarie,AU,70.00,scattered clouds,-31.43,152.92,Rydges Port Macquarie
3,Byron Bay,AU,75.20,scattered clouds,-28.65,153.62,Backpackers Inn on the Beach
4,Biloela,AU,75.99,overcast clouds,-24.42,150.50,Silo Motor Inn


In [93]:
#Create map with the round trip directions - Driving
import gmaps.datasets
fig = gmaps.figure()
#Set directions
round_trip = gmaps.directions_layer(sydney, sydney,waypoints=[charlestown,port_macquarie,
                                                              byron_bay,biloela],
        travel_mode='DRIVING')
#Add lasyer
fig.add_layer(round_trip)
#Print
fig


Figure(layout=FigureLayout(height='420px'))

In [94]:
#Create map with the pop up marker
import gmaps.datasets
fig = gmaps.figure()

#Link locations
locations=round_trip_df[['Lat','Lng']]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info=[info_box_template.format(**row) for index, row in round_trip_df.iterrows()]
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)

#Add layer
fig.add_layer(marker_layer)

#Print
fig


Figure(layout=FigureLayout(height='420px'))